In [382]:
import pandas as pd

In [383]:
# Read Data
df = pd.read_csv('./بيانات السجلات التجارية لعام 2018 .xlsx - Sheet1.csv')

/tmp/ipykernel_53196/1480514459.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./بيانات السجلات التجارية لعام 2018 .xlsx - Sheet1.csv')


# EDA

In [384]:
# df.describe(include='all').T

In [385]:
# df['ID'].value_counts().sum()
# df.describe(include='all').T

In [405]:
import pandas as pd
import re

# Function to normalize Arabic text by standardizing characters
def normalize_arabic(text):
    if pd.isna(text):
        return ""
    text = str(text)
    # Normalize alef variations (أ/ا/إ/آ)
    text = re.sub('[أإآ]', 'ا', text)
    # Normalize taa marbuta and haa (ة/ه)
    text = re.sub('ة', 'ه', text)
    # Normalize yaa and alif maksura (ي/ى)
    text = re.sub('ى', 'ي', text)
    return text

# Function to determine the business sector based on keywords
def identify_sector(text):
    if pd.isna(text):
        return 'أخرى'
    
    # Normalize the text first
    normalized_text = normalize_arabic(str(text))
# Dictionary of sectors with updated keywords
    sectors = {
        # 1. Food and Beverage (المطاعم والمقاهي)
        'المطاعم والمقاهي': [
            'مطعم', 'بوفيه', 'عصاير', 'عصير', 'فوال', 'مطابخ', 'بخاري', 'اكلات', 'وجبات',
            'فلافل', 'مقهي', 'محامص', 'قهوه', 'مشروبات', 'مثلجات', 'حلويات', 'خبز', 'مخبز',
            'معجنات', 'مخابز', 'كافيه', 'كوكيز', 'فطاير', 'مندي', 'شاورما', 'سناكس', 'تجهيز',
            'كفتيريا', 'كافتريا', 'فول', 'تمر', 'طاحون', 'لقيمات', 'خباز', 
            'كافتيريا', 'فرن', 'مطبخ', 'ولائم', 'اعاشه', 'فطائر', 'ساندويتش', 'مضغوط', 'كوفي', 
            'شاي', 'ايسكريم'
        ],

        # 2. Retail Food (تجارة المواد الغذائية)
        'تجارة المواد الغذائية': [
            'بقاله', 'تموينات', 'غذائيه','غذاييه', 'مستودع', 'لحوم', 'ملحمه', 'دواجن', 'خضار',
            'فواكه', 'مكسرات', 'اسماك', 'دقيق', 'عسل', 'اعلاف', 'مواشي', 'مطاحن', 'مسمكه',
            'صيد', 'مكملات', 'بهارات', 'خضراء', 'توابل', 'ذبايح', 'اطعمه',
            'مؤونه', 'بستان', 'اغذيه', 'معاصر', 
            'غذائيه', 'محل', 'خضروات', 'مياه', 'تحليه', 'تعبئه', 'منتجات', 'معدنية', 'عصائر', 
            'مشروبات', 'تمور', 'مزاد', 'عسل', 'لحوم', 'اسماك', 'تموينات', 
            'سوبرماركت', 'حبوب'
        ],

        # 3. Healthcare (الخدمات الصحية)
        'الخدمات الصحية': [
            'صيدليه', 'ادويه', 'طبي', 'طبيه', 'اسنان', 'بيطريه', 'عيون', 'مختبر', 'صحه',
            'عيادات', 'بصريات', 'حجامه', 'مساج', 'نفسيه', 'عصبيه', 'نقاهه', 'تاهيل',
            'عياده', 'مستشفى', 'نقاهه', 'رعايه'
        ],

        # 4. Automotive (خدمات السيارات)
        'خدمات السيارات': [
            'محطه', 'محروقات', 'وقود', 'غاز', 'ورشه', 'صيانه', 'مغسله', 'سيارات',
            'بنشر', 'اطارات', 'زيوت', 'تشحيم', 'قطع', 'غيار', 'دراجات', 'ناريه', 'تاجير',
            'كهرباء', 'توجيه', 'مركبات', 'ميكانيكا','مکينيکا', 'اجره', 
            'فحص', 'سمكره', 'تشليح'
        ],

        # 5. Fashion and Beauty (الملابس والتجميل)
        'الملابس والتجميل': [
            'خياط', 'خياطه', 'ملابس', 'عباءات', 'ازياء', 'مشالح', 'رجاليه', 'اطفال', 'اقمشه',
            'زي', 'موحد', 'صالون', 'حلاقه', 'حلاق', 'عطور', 'ادوات', 'تجميل', 'مستحضرات',
            'تزيين', 'نسائي','نسايي', 'مشغل', 'زينه', 'تلميع', 'مغسله', 'غسيل', 'مغاسل', 'مفروشات',
            'ستاير', 'عسكريه', 'تفصيل', 'فساتين', 'اناقه', 
            'الستائر', 'بورسلان', 'شنط', 'حقائب','حقايب', 'مجوهرات', 'نحاسيات',  'اكسسوارات', 
            'عبايات',
        ],

        # 6. Construction and Real Estate (البناء والعقارات)
        'البناء والعقارات': [
            'مقاولات', 'بناء','صناعه', 'اسمنتيه', 'ديكور', 'تصميم', 'داخلي', 'تكييف',
            'تبريد', 'تشطيب', 'ابراج', 'هياكل', 'حداده', 'نجاره', 'المنيوم', 'صناعات', 'خشبيه',
            'موبيليا', 'تنجيد', 'اثاث', 'رخام', 'حجر', 'انابيب', 'مصاعد', 'سراميك', 'خرده',
            'سكراب', 'مفاتيح', 'حفر', 'ابار', 'عقارات', 'عقاريه', 'تقييم', 'عقاري',
            'تطوير', 'استثمار', 'عقار', 'اداره', 'املاك','املاک', 'هندسه', 'معماريه', 'وساطه', 'اسكان',
            'مصلح', 
            'دهانات', 'ورق الجدران', 'عزل', 'حراري', 'بلاستيك', 'رخام', 'المنيوم', 'حداد', 
            'نجاره', 'تشطيب', 'مقاول', 'عقارات', 'مساحه', 'ارضي',
            'ابواب ','اوتوماتيكيه', 'ترميم', 'تعمير', 'اسفلت', 'مصعد', 'تكييف','تكيف'
        ],

        # 7. Technology and Electronics (التقنية والإلكترونيات)
        'التقنية والإلكترونيات': [
            'اتصالات', 'جوالات', 'الكترونيات', 'اجهزه', 'تقنيه', 'شبكات', 'حاسب',
            'حلول', 'ذكيه', 'ادوات', 'کهربائيه','کهرباييه', 'منزليه', 'اکترونيات', 
            'نشر', 'الكتروني', 'برمج', 'مطور', 'صوتيات', 'كهرباء', 
            'اكترونيات',
            'طاقه', 'متجدده', 'نوويه','للطاقه',

        ],

        # 8. General Trade (التجارة العامة)
        'التجارة العامة': [
            'تجاره', 'تجاريه', 'جمله', 'مجموعه', 'دوليه', 'محدوده', 'استيراد', 'تصدير', 'متجر',
            'مجمع', 'شركه', 'قابضه', 'عالميه', 
            'تجزئة', 'سوق', 'افتراضي', 'محل', 'مزاد', 'تجاري', 
            'دكان', 'كماليات',
        ],

        # 9. Specialized Retail (التجارة المتخصصة)
        'التجارة المتخصصة': [
            'هدايا', 'کوش', 'تغليف', 'قرطاسيه', 'تحف', 'مفارش', 'ذهب', 'معادن', 'ثمينه', 'الماس',
            'احذيه', 'حقايب', 'شنط', 'قطع', 'غيار', 'لوازم', 'رحلات', 'نظارات', 'زجاج', 'زهور',
            'بطاريات', 'کماليات', 'مناسبات', 'ساعات', 'سجاد', 'اواني', 'مظلات', 'خيام', 'طيور',
            'حيوانات', 'اليفه', 'بلاستيک', 'عطاره', 'بلاستيكيه', 'منتجات', 'جلود', 'كيماويات',
            'مسلخ', 
            'خزانات', 'فحم', 'عود', 'بازارات', 'احجار', 'كريمة', 'خشب', 'نباتات', 
            'مفروشات', 'ستائر', 'تحف', 'قرطاسية', 'كتب', 'مكتبة', 'ألعاب', 'أحبار', 
            
        ],

        # 10. Maintenance Services (خدمات الصيانة والتنظيف)
        'خدمات الصيانة والتنظيف': [
            'صيانه', 'نظافه', 'تخزين', 'اصلاح', 'الات', 'تشغيل', 
            'تنظيف', 'تحليه', 'مياه', 'تعبئة', 'لوجستية', 'نقل', 'شحن', 'صرف','صحي','منظفات'
        ],

        # 11. Business Services (الخدمات المهنية والإدارية)
        'الخدمات المهنية والإدارية': [
            'استشارات', 'صناعيه', 'اداريه', 'محاماه', 'ترجمه', 'تنظيم', 'معارض', 'موتمرات',
            'تسويق', 'دعايه', 'اعلان', 'عامه', 'اعمال', 'انتاج', 'فني', 'اعلامي',
            'مرئي','مريي', 'مسموع', 'تصوير', 'فوتوغرافي', 'نسايي','نسخ', 'طباعه', 'موارد',
            'بشريه', 'توظيف', 'اهلي', 'فنيه', 'تحصيل', 'ديون', 'استقدام', 'مشاريع', 'خط',
            'رسم', 'مراجعه', 'تدقيق', 'خراطه', 'محامون', 'مستشارون', 'مؤتمرات', 'تمويل', 'صرافه',
            'تقسيط', 'بحوث', 'دراسات', 
            'مكتبه','مكتب',  'تخليص', 'جمركي', 'مساحة', 'ارضية', 'محامي', 'استشارات', 
            'تدقيق', 'مزادات', 'تحكيم', 'تصميم', 'استشارات', 'هندسة'
        ],

        # 12. Travel and Tourism (السفر والسياحة)
        'السفر والسياحة': [
            'سفر', 'سياحه', 'وکاله', 'عطلات', 'تنظيم', 'رحلات', 'وحدات', 'سکنيه', 'شقق',
            'مفروشه', 'تاجير', 'لوازم', 'مناسبات', 'افراح', 'احتفالات', 'فندق', 'فنادق', 
            'نزل', 'سياحي', 'ايواء', 'منتجع', 'رحلات', 'فنادق', 'استراحه', 'إقامه'
        ],

        # 13. Entertainment and Recreation (الترفيه والتسلية)
        'الترفيه والتسلية': [
            'ترفيه', 'العاب', 'اطفال', 'فعاليات', 'رياضي', 'مرکز', 'موسيقى', 'ملاعب', 
            'فنون', 'موسيقي', 'حفلات', 'العاب', 'سباقات', 'استديو', 'تسجيلات'
        ],

        # 14. Education and Training (التعليم والتدريب)
        'التعليم والتدريب': [
            'تدريب', 'اکاديميه', 'طلابيه', 'لغات', 'مدرسي', 'رياض', 'اطفال',   
            'قياده', 'مدارس', 'تعليم', 'مركز', 'معرف', 'معهد', 'وسايل', 'شرعيه', 'مبرمج', 
            'جامعه', 'تعليمي', 'قراءه', 'تدريس', 'أكاديميه', 'مدرسه', 'روضه','تدريس','مدارس','أكاديميه',
        ],

        # 15. Manufacturing (التصنيع والإنتاج)
        'التصنيع والإنتاج': [
            'مصنع', 'انتاج', 'تصنيع', 'منجرة', 
            'تدوير', 'نفايات', 'بلاستيك', 'كرتون', 'منتجات', 'صناعات', 'أرضيات', 'معاصر', 
            'معمل', 'كسارة', 'معصرة', 'منجرة', 'حدادة'
        ],

        # 16. Logistics and Transportation (النقل والخدمات اللوجستية)
        'النقل والخدمات اللوجستية': [
            'نقليات', 'لوجستيه', 'نقل', 'تخزين', 'بري', 'بحري', 'بضائع','بضايع', 
            'جوي', 'طرود', 'تموين', 
            'لوجستي', 'شحن',  'طيران','طرود','نقليات'
        ],

        # 17. Agriculture (الزراعة)
        'الزراعة': [
            'انتاج', 'زراعي', 'زراعه', 'زراعيه', 'خيول', 'مربط', 
            'مزرعه', 'نباتات', 'شتله', 'خيول', 'زراعة', 'اسمده','محاصيل','زراعي','شتله',
        ],

        # 18. Security Services (خدمات الأمن والسلامة)
        'خدمات الأمن والسلامة': [
            'مراقبه', 'امنيه', 'انظمه',  'ادوات', 'اجهزه', 'حراسات','حرسات', 
            'حماية', 'أمن', 'سلامه', 'حمايه'
        ],

        # 19. Equipment Rental (تأجير المعدات)
        'تأجير المعدات': [
            'صيانه', 'الات', 'ادوات', 
            'معدات',
        ],

        # 20. Charitable Organizations (المؤسسات الخيرية والأهلية)
        'المؤسسات الخيرية والأهلية': [
            'خيريه', 'وقف', 'اوقاف', 'جمعيه', 'وصيه', 'لجنه', 'اهليه', 'احتياجات', 'خاصه', 
            'خيري', 'توعيه', 'انقاذ', 'ايتام', 'جمعية',
            'ارشاد', 'اسري', 'اجتماعيه', 
            'اسره', 'اجتماعي', 'رعايه', 'نفسيه'
        ],
    }
    # Check for matches in each category
    for sector, keywords in sectors.items():
        for keyword in keywords:
            # Use normalized keywords for matching
            if keyword in normalized_text:
                return sector
    
    # If no match is found
    return 'أخرى'

# Main execution
if __name__ == "__main__":
    # Add new column with predicted categories
    df['القطاع'] = df['اسم النشاط التجاري '].apply(identify_sector)
    
    # Display the result
    print(df)

                  ID  رقم السجل التجاري  نوع السجل التجاري   \
0       7.002085e+09          1010930458              رئيسي   
1       7.002101e+09          1010930562              رئيسي   
2       7.002085e+09          1010930461              رئيسي   
3       7.002085e+09          1010930457              رئيسي   
4       7.002091e+09          1010930510              رئيسي   
...              ...                 ...                ...   
172523           NaN          4031078107               فرعي   
172524           NaN          2251068444               فرعي   
172525           NaN          5950034588               فرعي   
172526           NaN          4030295947               فرعي   
172527           NaN          2051064705               فرعي   

                                      اسم النشاط التجاري   \
0       مؤسسة خالد احمد مبارك الحمود لبيع الجوالات وصي...   
1                                  مؤسسة ركن لارا للتجارة   
2            مؤسسة محليه صالح بن ضيف الله اليامي التجارية   